<a href="https://colab.research.google.com/github/CamiloVga/Curso-IA-Para-Ciencia-de-Datos/blob/main/Script_Sesi%C3%B3n_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IA para la Ciencia de Datos
## Universidad de los Andes

**Profesor:** Camilo Vega - AI/ML Engineer  
**LinkedIn:** https://www.linkedin.com/in/camilo-vega-169084b1/

---

## Guía: Sistemas de Agentes Inteligentes y Multi-Agente con LLM

Este notebook presenta **3 implementaciones prácticas**:

1. **RAG Agent con Langchain + OpenAI** - Agente inteligente que busca en documentos locales
2. **Deep Research Agent con Groq + Tavily** - Investigación web con trazabilidad de fuentes
3. **Sistema Multi-Agente con CrewAI** - Flujo editorial completo (4 agentes especializados)

### Arquitecturas de Agentes
- **Agente Simple:** Un LLM + herramientas específicas + prompt estructurado
- **Agente de Investigación:** LLM + búsqueda web + gestión de fuentes
- **Multi-Agente:** Múltiples LLMs especializados trabajando en pipeline colaborativo

### Requisitos
- **APIs Principales:**
  - OpenAI API token (GPT-4o, GPT-4o-mini)
  - Groq API token (Llama, Mixtral, otros)
  - Tavily API token (búsqueda web especializada)
- **GPU:** No requerida (uso de APIs)
- **Datos:** Documentos PDF para RAG local (carpeta `/content/carpeta_rag`)

## Configuración APIs
- **OpenAI API:**
  1. [Crear token](https://platform.openai.com/api-keys)
  2. En Colab: 🔑 Secrets → Agregar `OPENAI_API_KEY` → Pegar tu token

- **Groq API:**
  1. [Crear token](https://console.groq.com/keys)
  2. En Colab: 🔑 Secrets → Agregar `GROQ_KEY` → Pegar tu token

- **Tavily API:**
  1. [Crear token](https://tavily.com)
  2. En Colab: 🔑 Secrets → Agregar `TAVILY_KEY` → Pegar tu token

### Frameworks y Librerías
- **LangChain:** Orquestación de agentes y herramientas
- **CrewAI:** Sistema multi-agente con roles y tareas específicas
- **ChromaDB:** Base de datos vectorial para RAG
- **Tavily:** API de búsqueda web optimizada para IA

### Casos de Uso Implementados
1. **Consultor de Documentos:** RAG que responde preguntas sobre archivos PDF locales
2. **Investigador Digital:** Búsqueda web inteligente con referencias y fuentes
3. **Redacción Editorial:** 4 agentes (Investigador + Redactor + Editor + Community Manager) trabajando en pipeline

Cada sección es **independiente** y puede ejecutarse por separado. Los agentes pueden configurarse con diferentes modelos de LLM según necesidades de velocidad vs. calidad.

#Agente RAG-Langchain

In [ ]:
#RAG Langchain + OpenAI

# 0. Instalaciones
!pip install langchain langchain-openai langchain-community chromadb pypdf

# 1. Imports
import os
from google.colab import userdata
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.tools import Tool
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 2. Configuración API
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# 3. Crear herramienta RAG
def create_rag_search():
    # Cargar documentos
    loader = DirectoryLoader("/content/carpeta_rag", glob="**/*.pdf", loader_cls=PyPDFLoader) #Crear carpeta en el directorio llamada carpeta_rag y subir los PDF a analizar
    docs = loader.load()

    # Dividir en chunks
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = splitter.split_documents(docs)

    # Crear vectorstore
    vectorstore = Chroma.from_documents(splits, OpenAIEmbeddings())
    retriever = vectorstore.as_retriever()

    # Función de búsqueda
    def search_docs(query: str) -> str:
        docs = retriever.get_relevant_documents(query)
        return "\n".join([doc.page_content for doc in docs[:2]])

    return Tool(
        name="search_documents",
        description="Busca información en los documentos locales",
        func=search_docs
    )

# 4. Configurar herramientas disponibles
rag_search = create_rag_search()
tools = [rag_search]

# 5. Crear prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful research assistant"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# 6. Crear agente con herramientas
agent = create_openai_functions_agent(
    llm=ChatOpenAI(),
    tools=tools,
    prompt=prompt
)

# 7. Ejecutor del agente
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # Dejar verbose=False para que no muestre el proceso interno del agente para responder

# 8. Ejemplo de uso
response = agent_executor.invoke({"input": "¿De qué tratan los documentos?"})
print(response['output'])

#Agente DeepResearch (Internet)-Langchain

In [ ]:
# Deep Research Agent + OpenAI

# 0. Instalaciones
!pip install langchain langchain-openai tavily-python

# 1. Imports
import os
from google.colab import userdata
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from tavily import TavilyClient

# 2. Configuración API
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
tavily_client = TavilyClient(api_key=userdata.get('TAVILY_KEY'))

# Configuración OpenAI
OPENAI_MODEL = "gpt-4o"  # Cambiar por: "gpt-4o", "gpt-4-turbo", "gpt-3.5-turbo"
TEMPERATURE = 0.1        # Creatividad (0.0=determinista, 1.0=creativo)

# Configuración Tavily
MAX_RESULTS = 5          # Número de resultados por búsqueda (1-20)
TOPIC = "general"        # Tipo: "general", "news", "finance"
# TIME_RANGE = "day"     # Tiempo: "day", "week", "month", "year"
# LANGUAGE = "es"        # Idioma preferido
# LOCATION = "CO"        # Código de país (CO=Colombia)
# INCLUDE_ANSWER = "basic"  # Respuesta: "basic", "advanced"
# INCLUDE_RAW_CONTENT = False  # True para contenido completo
# INCLUDE_DOMAINS = ["ejemplo.com"]  # Dominios específicos
# EXCLUDE_DOMAINS = ["spam.com"]     # Excluir dominios

# 3. Crear herramienta de investigación web
def create_web_search():
    def search_web(query: str) -> str:
        """Busca información actualizada en la web"""
        try:
            # Configuración básica de búsqueda
            search_params = {
                "query": query,
                "max_results": MAX_RESULTS,
                "topic": TOPIC
            }

            # Agregar parámetros opcionales (descomenta según necesites)
            # if TIME_RANGE: search_params["time_range"] = TIME_RANGE
            # if LANGUAGE: search_params["language"] = LANGUAGE
            # if LOCATION: search_params["location"] = LOCATION
            # if INCLUDE_ANSWER: search_params["include_answer"] = INCLUDE_ANSWER
            # if INCLUDE_RAW_CONTENT: search_params["include_raw_content"] = INCLUDE_RAW_CONTENT
            # if INCLUDE_DOMAINS: search_params["include_domains"] = INCLUDE_DOMAINS
            # if EXCLUDE_DOMAINS: search_params["exclude_domains"] = EXCLUDE_DOMAINS

            response = tavily_client.search(**search_params)

            # Extraer contenido relevante
            results = []
            for result in response.get("results", []):
                title = result.get('title', 'Sin título')
                content = result.get('content', '')[:500]
                results.append(f"**{title}**\n{content}...")

            return "\n\n".join(results) if results else "No se encontró información relevante."

        except Exception as e:
            return f"Error en búsqueda web: {e}"

    return Tool(
        name="web_search",
        description="Busca información actualizada en la web sobre cualquier tema",
        func=search_web
    )

# 4. Configurar herramientas disponibles
web_search = create_web_search()
tools = [web_search]

# 5. Crear prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un investigador experto que analiza información y responde de manera clara y detallada. Siempre usa la herramienta de búsqueda web para obtener información actualizada."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# 6. Crear agente con herramientas
agent = create_openai_functions_agent(
    llm=ChatOpenAI(model=OPENAI_MODEL, temperature=TEMPERATURE),
    tools=tools,
    prompt=prompt
)

# 7. Ejecutor del agente
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # Dejar verbose=False para que no muestre el proceso interno del agente para responder

# 8. Ejemplo de uso
response = agent_executor.invoke({"input": "¿Quién es Camilo Vega Barbosa?"})
print(response['output'])

#Agente DeepResearch (Internet)-Groq

In [ ]:
# Deep Research Agent con Groq y Trazabilidad de Fuentes

# 1. Instalaciones
!pip install groq tavily-python

import os
from google.colab import userdata
from tavily import TavilyClient
from groq import Groq
from urllib.parse import urlparse

# 2. CONFIGURACIÓN

# Configuración Groq
GROQ_MODEL = "openai/gpt-oss-20b"        # Modelo de lenguaje a usar
MAX_TOKENS = 1024                        # Máximo tokens en respuesta (512-8192)
TEMPERATURE = 0.1                        # Creatividad (0.0=determinista, 1.0=creativo)

SYSTEM_PROMPT = """Eres un investigador experto. Analiza la información y responde de manera clara, detallada y objetiva.
IMPORTANTE: Incluye referencias [1], [2], etc. para cada información específica que uses. Ademas prioriza la información más reciente que provenga de medios o fuentes confiables del lugar o del tema"""

# Configuración Tavily
MAX_RESULTS = 5                          # Resultados por búsqueda (1-20)
TAVILY_TOPIC = "general"                 # Tipo: "general", "news", "finance"
TAVILY_TIME_RANGE = None                 # Tiempo: "day", "week", "month", "year"
TAVILY_INCLUDE_ANSWER = "basic"          # Respuesta: "basic", "advanced"
TAVILY_INCLUDE_RAW_CONTENT = False       # True para contenido completo
TAVILY_INCLUDE_DOMAINS = None            # ["ejemplo.com"] para dominios específicos
TAVILY_EXCLUDE_DOMAINS = None            # ["spam.com"] para excluir dominios
TAVILY_LOCATION = "CO"                   # Código de país (CO=Colombia)
TAVILY_LANGUAGE = "es"                   # Idioma preferido

# Configuración investigación
MAX_ADDITIONAL_SEARCHES = 2              # Búsquedas adicionales máximas
ADDITIONAL_RESULTS = 3                   # Resultados por búsqueda adicional
MAX_CONTENT_CHARS = 800                  # Caracteres por fuente

# 3. INICIALIZACIÓN
groq_client = Groq(api_key=userdata.get('GROQ_KEY'))
tavily = TavilyClient(api_key=userdata.get('TAVILY_KEY'))
print("✅ Clientes inicializados")

# 4. SISTEMA DE TRAZABILIDAD Y FUNCIONES PRINCIPALES

class SimpleTracker:
    """Gestiona el registro y formato de fuentes consultadas"""
    def __init__(self):
        self.sources = []
        self.counter = 0

    def add(self, title, url, content):
        """Registra una nueva fuente y retorna su ID de referencia"""
        self.counter += 1
        self.sources.append({
            'id': self.counter, 'title': title, 'url': url,
            'content': content[:200] + "..." if len(content) > 200 else content,
            'domain': urlparse(url).netloc if url else "Unknown"
        })
        return self.counter

    def format_sources(self):
        """Genera el formato final de fuentes para el reporte"""
        if not self.sources:
            return "No hay fuentes."

        result = f"\n## 📚 FUENTES ({len(self.sources)} consultadas)\n\n"
        domains = {}

        for s in self.sources:
            result += f"**[{s['id']}]** {s['title']}\n🔗 {s['url']}\n📄 {s['content']}\n\n"
            domains[s['domain']] = domains.get(s['domain'], 0) + 1

        result += "**Dominios:** " + ", ".join([f"{d}({c})" for d, c in domains.items()])
        return result

def buscar(query, tracker, max_results=MAX_RESULTS):
    """Búsqueda web con configuración completa y registro de fuentes"""
    try:
        # Configurar parámetros de búsqueda
        search_params = {
            "query": query,
            "max_results": max_results,
            "topic": TAVILY_TOPIC
        }

        # Agregar parámetros opcionales
        if TAVILY_TIME_RANGE:
            search_params["time_range"] = TAVILY_TIME_RANGE
        if TAVILY_INCLUDE_ANSWER:
            search_params["include_answer"] = TAVILY_INCLUDE_ANSWER
        if TAVILY_INCLUDE_RAW_CONTENT:
            search_params["include_raw_content"] = TAVILY_INCLUDE_RAW_CONTENT
        if TAVILY_INCLUDE_DOMAINS:
            search_params["include_domains"] = TAVILY_INCLUDE_DOMAINS
        if TAVILY_EXCLUDE_DOMAINS:
            search_params["exclude_domains"] = TAVILY_EXCLUDE_DOMAINS
        if TAVILY_LOCATION:
            search_params["location"] = TAVILY_LOCATION
        if TAVILY_LANGUAGE:
            search_params["language"] = TAVILY_LANGUAGE

        response = tavily.search(**search_params)
        content = ""

        for r in response.get("results", []):
            source_id = tracker.add(
                r.get('title', 'Sin título'),
                r.get('url', ''),
                r.get('content', '')
            )
            content += f"**[{source_id}] {r.get('title', '')}**\n"
            content += f"{r.get('content', '')[:MAX_CONTENT_CHARS]}...\n\n"

        return content
    except Exception as e:
        return f"Error: {e}"

def generar(prompt):
    """Generación de análisis usando Groq"""
    try:
        response = groq_client.chat.completions.create(
            model=GROQ_MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": prompt}
            ],
            max_tokens=MAX_TOKENS,
            temperature=TEMPERATURE
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

def investigar(tema, busquedas_extra=None):
    """Investigación completa con múltiples búsquedas y trazabilidad"""
    print(f"🔍 Investigando: {tema}")
    tracker = SimpleTracker()

    # Búsqueda principal
    info = buscar(tema, tracker)

    # Búsquedas adicionales
    if busquedas_extra:
        for query in busquedas_extra[:MAX_ADDITIONAL_SEARCHES]:
            print(f"📡 Búsqueda adicional: {query}")
            info += f"\n--- {query} ---\n" + buscar(query, tracker, ADDITIONAL_RESULTS)

    # Generar reporte
    print("🤖 Generando análisis...")
    prompt = f"""Analiza la siguiente información sobre: {tema}

Estructura tu respuesta con:
1. Resumen ejecutivo
2. Hallazgos clave
3. Análisis y tendencias
4. Conclusiones

Incluye referencias [1], [2], etc. para información específica.

INFORMACIÓN:
{info}"""

    response = generar(prompt)

    # Reporte final
    return f"# 📋 {tema.upper()}\n\n{response}\n\n{'='*60}\n{tracker.format_sources()}"

# 5. USO SIMPLE

# Investigación en 1-2 líneas:
reporte = investigar("¿Quién es Camilo Vega Barbosa?")
print(reporte)

#Sistema MultiAgente-CrewAI

In [ ]:
#Sistema MultiAgente con CrewAI-Flujo de trabajo de equipo en un periódico

# Instalaciones necesarias
!pip install crewai tavily-python openai langchain-openai

import os
from google.colab import userdata
from tavily import TavilyClient
from openai import OpenAI
from crewai import Agent, Task, Crew
from crewai.tools import tool
from langchain_openai import ChatOpenAI
from urllib.parse import urlparse
import re

# =============================================================================
# CONFIGURACIÓN GLOBAL
# =============================================================================

# Configurar OpenAI API Key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Modelos OpenAI para cada agente (usando ChatGPT-4o-mini)
OPENAI_MODELS = {
    "research": "gpt-4o-mini",     # Investigación
    "writing": "gpt-4o-mini",      # Escritura
    "editing": "gpt-4o-mini",      # Edición
    "social": "gpt-4o-mini"        # Social media
}

# Configuración Tavily
TAVILY_CONFIG = {
    "max_results": 5,
    "topic": "general",
    "location": "CO",
    "language": "es",
    "include_answer": "basic"
}

# =============================================================================
# HERRAMIENTAS Y TRACKER
# =============================================================================

# Variable global para el tracker
global_tracker = None

class NewsroomTracker:
    """Gestiona fuentes y referencias"""
    def __init__(self):
        self.sources = []
        self.counter = 0

    def add_source(self, title, url, content, agent_name):
        self.counter += 1
        self.sources.append({
            'id': self.counter,
            'title': title,
            'url': url,
            'content': content[:200] + "..." if len(content) > 200 else content,
            'domain': urlparse(url).netloc if url else "Unknown",
            'found_by': agent_name
        })
        return self.counter

    def get_sources_summary(self):
        if not self.sources:
            return "Sin fuentes disponibles"

        summary = f"\n📚 FUENTES CONSULTADAS ({len(self.sources)})\n" + "="*50 + "\n"
        for s in self.sources:
            summary += f"[{s['id']}] {s['title']} | {s['domain']}\n"
            summary += f"🔗 {s['url']}\n"
            summary += f"📄 {s['content']}\n\n"
        return summary

@tool("search_web")
def search_web_tool(query: str) -> str:
    """Busca información en web usando Tavily API - LIMITADO A 2 RESULTADOS"""
    global global_tracker
    try:
        tavily = TavilyClient(api_key=userdata.get('TAVILY_KEY'))

        # Forzar explícitamente max_results = 2
        search_params = {
            "query": query,
            "max_results": 2,  # FORZADO A 2
            "topic": "general",
            "location": "CO",
            "language": "es",
            "include_answer": "basic"
        }

        print(f"🔍 Ejecutando búsqueda con max_results = {search_params['max_results']}")
        response = tavily.search(**search_params)

        results = response.get("results", [])
        print(f"✅ Búsqueda: '{query}' | Resultados obtenidos: {len(results)} | Máximo configurado: {search_params['max_results']}")

        content = f"🔍 Resultados para: {query} (máx: {search_params['max_results']})\n\n"
        for i, result in enumerate(results, 1):
            source_id = global_tracker.add_source(
                result.get('title', 'Sin título'),
                result.get('url', ''),
                result.get('content', ''),
                'Investigador'
            )
            content += f"**[{source_id}] Resultado {i}/{len(results)}: {result.get('title', '')}**\n"
            content += f"{result.get('content', '')[:400]}...\n\n"

        return content
    except Exception as e:
        return f"Error en búsqueda: {e}"

# =============================================================================
# CONFIGURACIÓN DE AGENTES
# =============================================================================

def setup_newsroom_agents():
    """Configura los 4 agentes especializados con OpenAI"""

    # Configurar LLMs con ChatOpenAI
    llms = {}
    for role, model in OPENAI_MODELS.items():
        llms[role] = ChatOpenAI(
            model=model,
            temperature=0.1 if role in ['research', 'editing'] else 0.7,
            api_key=userdata.get('OPENAI_API_KEY')
        )
        print(f"✅ Modelo OpenAI {model} configurado para {role}")

    # 1. AGENTE INVESTIGADOR
    research_agent = Agent(
        role='Investigador Senior',
        goal='Encontrar información completa y verificada sobre cualquier tema',
        backstory="""Eres un investigador periodístico experto con 15 años de experiencia.
        Sabes distinguir fuentes confiables de rumores y encontrar información precisa.
        Tu especialidad es buscar datos verificados y contexto histórico relevante.""",
        verbose=True,
        allow_delegation=False,
        llm=llms['research'],
        tools=[search_web_tool]
    )

    # 2. AGENTE REDACTOR
    writing_agent = Agent(
        role='Redactor Principal',
        goal='Crear artículos periodísticos claros, informativos y atractivos',
        backstory="""Eres un redactor senior especializado en periodismo digital.
        Escribes con estilo claro y directo, estructuras información de manera lógica
        y siempre incluyes referencias apropiadas a las fuentes consultadas.""",
        verbose=True,
        allow_delegation=False,
        llm=llms['writing']
    )

    # 3. AGENTE EDITOR
    editing_agent = Agent(
        role='Editor Jefe',
        goal='Revisar y mejorar la calidad editorial del contenido',
        backstory="""Eres el editor jefe con 20 años de experiencia en medios digitales.
        Tu ojo crítico detecta inconsistencias, errores factuales y problemas de estilo.
        Garantizas que cada publicación cumpla los estándares editoriales más altos.""",
        verbose=True,
        allow_delegation=False,
        llm=llms['editing']
    )

    # 4. AGENTE REDES SOCIALES
    social_agent = Agent(
        role='Community Manager',
        goal='Crear contenido viral y engaging para redes sociales',
        backstory="""Eres especialista en redes sociales que entiende las tendencias digitales.
        Sabes crear hooks atractivos, usar hashtags estratégicos y adaptar mensajes
        para cada plataforma. Tu contenido genera engagement y conversación.""",
        verbose=True,
        allow_delegation=False,
        llm=llms['social']
    )

    return research_agent, writing_agent, editing_agent, social_agent

# =============================================================================
# DEFINICIÓN DE TAREAS
# =============================================================================

def create_newsroom_tasks(agents, topic):
    """Crea las tareas para el flujo de trabajo"""

    research_agent, writing_agent, editing_agent, social_agent = agents

    # TAREA 1: INVESTIGACIÓN PROFUNDA
    research_task = Task(
        description=f"""
        Investiga a fondo sobre: {topic}

        Debes realizar múltiples búsquedas para cubrir:
        1. Información básica y contexto general
        2. Datos específicos y cifras actualizadas
        3. Diferentes perspectivas del tema
        4. Fuentes oficiales y expertos en el tema
        5. Contexto histórico si es relevante

        Para cada búsqueda, analiza los resultados y extrae:
        - Información clave con referencias [1], [2], etc.
        - Datos verificables y fechas importantes
        - Diferentes puntos de vista
        - Fuentes primarias cuando sea posible

        Entrega un informe estructurado con:
        - Resumen ejecutivo (2-3 párrafos)
        - Hallazgos principales organizados por temas
        - Referencias numeradas a todas las fuentes
        - Conclusiones preliminares
        """,
        agent=research_agent,
        expected_output="Informe de investigación detallado con múltiples fuentes verificadas"
    )

    # TAREA 2: REDACCIÓN DEL ARTÍCULO
    writing_task = Task(
        description=f"""
        Basándote en la investigación completa, redacta un artículo periodístico profesional sobre: {topic}

        El artículo debe incluir:

        1. **TITULAR**: Atractivo, preciso y que capte la atención
        2. **LEAD** (primer párrafo): Responde las 5W - qué, quién, cuándo, dónde, por qué
        3. **DESARROLLO**:
           - Estructura de pirámide invertida (información más importante primero)
           - Subtítulos para facilitar la lectura
           - Párrafos cortos (máximo 3 líneas cada uno)
           - Citas y referencias numeradas [1], [2], etc.
        4. **CONCLUSIÓN**: Resumen de puntos clave y perspectivas futuras

        **Estilo periodístico:**
        - Lenguaje claro, directo y objetivo
        - Voz activa
        - Información verificable y atribuida
        - Tono profesional pero accesible
        - Transiciones fluidas entre párrafos

        **Extensión**: 600-900 palabras
        **Referencias**: Incluir todas las fuentes como [1], [2], etc.
        """,
        agent=writing_agent,
        expected_output="Artículo periodístico completo y bien estructurado",
        context=[research_task]
    )

    # TAREA 3: EDICIÓN Y CORRECCIÓN
    editing_task = Task(
        description=f"""
        Revisa meticulosamente y mejora el artículo sobre: {topic}

        **VERIFICACIÓN EDITORIAL:**
        1. **Precisión factual**: Confirma datos, fechas y cifras
        2. **Gramática y ortografía**: Corrección completa
        3. **Estilo y claridad**: Mejora la fluidez del texto
        4. **Estructura**: Verifica lógica y coherencia
        5. **Referencias**: Asegura uso correcto de fuentes [1], [2], etc.
        6. **Estándares editoriales**: Cumplimiento de normas periodísticas

        **MEJORAS A IMPLEMENTAR:**
        - Fortalece el titular si es necesario
        - Mejora las transiciones entre párrafos
        - Optimiza la claridad de las ideas
        - Verifica que el lead responda todas las 5W
        - Asegura balance entre información y legibilidad

        **ENTREGA:**
        1. Artículo final editado y corregido
        2. Lista detallada de todos los cambios realizados
        3. Justificación de las mejoras implementadas
        4. Recomendaciones adicionales si las hay
        """,
        agent=editing_agent,
        expected_output="Artículo final editado + lista completa de mejoras implementadas",
        context=[writing_task]
    )

    # TAREA 4: CONTENIDO PARA REDES SOCIALES
    social_task = Task(
        description=f"""
        Crea una estrategia completa de contenido para redes sociales basada en el artículo sobre: {topic}

        **CONTENIDO A GENERAR:**

        1. **TWITTER/X** (3 tweets diferentes):
           - Tweet 1: Hook + dato impactante (max 280 caracteres)
           - Tweet 2: Perspectiva diferente + pregunta (max 280 caracteres)
           - Tweet 3: Call-to-action + enlace (max 280 caracteres)

        2. **FACEBOOK**:
           - Post engaging con pregunta para generar interacción
           - Incluye emoji relevante y call-to-action
           - 100-150 palabras

        3. **LINKEDIN**:
           - Post profesional con insights clave
           - Enfoque en implicaciones business/profesionales
           - 150-200 palabras

        4. **INSTAGRAM STORIES**:
           - Texto corto y visual para historia
           - Incluye pregunta interactiva
           - 30-40 palabras máximo

        5. **HASHTAGS ESTRATÉGICOS**:
           - 8-12 hashtags relevantes y trending
           - Mix de hashtags populares y nicho
           - Incluye hashtags locales (Colombia)

        **ESTRATEGIA ADICIONAL:**
        - Mejores horarios para publicar en cada plataforma
        - Tipo de imágenes/videos recomendados
        - Estrategia de engagement y respuesta
        - KPIs a monitorear
        """,
        agent=social_agent,
        expected_output="Estrategia completa de redes sociales con contenido para múltiples plataformas",
        context=[editing_task]
    )

    return [research_task, writing_task, editing_task, social_task]

# =============================================================================
# FUNCIÓN PRINCIPAL
# =============================================================================

def ejecutar_redaccion_digital(tema):
    """Ejecuta el flujo completo de la redacción digital"""

    print(f"🏢 INICIANDO REDACCIÓN DIGITAL CON OPENAI")
    print(f"📰 Tema: {tema}")
    print(f"🤖 Modelo: ChatGPT-4o-mini")
    print("="*60)

    # Inicializar tracker global
    global global_tracker
    global_tracker = NewsroomTracker()

    # Configurar agentes
    print("⚙️ Configurando agentes...")
    agents = setup_newsroom_agents()

    # Crear tareas
    print("📋 Creando tareas...")
    tasks = create_newsroom_tasks(agents, tema)

    # Configurar crew
    print("🎯 Configurando crew...")
    newsroom_crew = Crew(
        agents=list(agents),
        tasks=tasks,
        verbose=True
    )

    # Ejecutar flujo de trabajo
    print("🚀 Ejecutando flujo de trabajo...")
    print("   1️⃣ Investigación en curso...")
    print("   2️⃣ Redacción en espera...")
    print("   3️⃣ Edición en espera...")
    print("   4️⃣ Social media en espera...")
    print("-"*60)

    # Ejecutar y capturar resultados
    crew_result = newsroom_crew.kickoff()

    # Obtener resultados individuales de cada tarea
    task_results = {}
    if hasattr(crew_result, 'tasks_output') and crew_result.tasks_output:
        for i, task_output in enumerate(crew_result.tasks_output):
            task_names = ['investigacion', 'articulo', 'edicion', 'redes_sociales']
            if i < len(task_names):
                task_results[task_names[i]] = task_output.raw if hasattr(task_output, 'raw') else str(task_output)
    else:
        # Fallback: intentar acceder a las tareas directamente
        for i, task in enumerate(tasks):
            task_names = ['investigacion', 'articulo', 'edicion', 'redes_sociales']
            if i < len(task_names) and hasattr(task, 'output'):
                task_results[task_names[i]] = task.output.raw if hasattr(task.output, 'raw') else str(task.output)

    # Si no se pudieron obtener resultados individuales, usar el resultado final
    if not task_results:
        task_results['redes_sociales'] = str(crew_result)

    # Generar reporte final completo
    final_report = f"""
# 📰 PRODUCCIÓN EDITORIAL COMPLETA
## Tema: {tema}
## Modelo: ChatGPT-4o-mini (OpenAI)
{'='*80}

## 🔍 1. INVESTIGACIÓN REALIZADA
{task_results.get('investigacion', 'No disponible')}

{'='*60}

## ✍️ 2. ARTÍCULO REDACTADO
{task_results.get('articulo', 'No disponible')}

{'='*60}

## 📝 3. ARTÍCULO EDITADO
{task_results.get('edicion', 'No disponible')}

{'='*60}

## 📱 4. CONTENIDO PARA REDES SOCIALES
{task_results.get('redes_sociales', str(crew_result))}

{'='*80}
{global_tracker.get_sources_summary()}

---
✅ **PROCESO COMPLETADO EXITOSAMENTE**
🤖 4 agentes especializados trabajaron en colaboración
📊 {len(global_tracker.sources)} fuentes consultadas y verificadas
🔗 **Pipeline ejecutado**: Investigación → Redacción → Edición → Social Media
⚡ **Modelo utilizado**: ChatGPT-4o-mini para máxima estabilidad
📋 **Tareas completadas**: {len([k for k, v in task_results.items() if v != 'No disponible'])}/4
"""

    return final_report

# =============================================================================
# EJEMPLO DE USO
# =============================================================================

if __name__ == "__main__":
    # Ejecutar con tema específico
    resultado = ejecutar_redaccion_digital("Nuevas regulaciones de IA en Colombia 2025")
    print(resultado)